This notebook represents an attempt to perform zero shot learning with the Gemma model.

# Installation of packages

Install transformers, we used the torch version.

In [2]:
!pip install transformers[torch]
#pip install transformers

Install datasets, which we ended up not relying on, but can be used as an alternative to attain datasets for text classification.

In [3]:
!pip install datasets

Install accelerate

In [5]:
!pip install -U accelerate

Install bitsandbytes

In [6]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:0000:0100:01


Install requests, which we used instead of datasets to get the IMDB data

In [7]:
!pip install requests

# Importing relevant libraries

In [8]:
import requests

In [9]:
import gzip

In [10]:
import shutil


In [11]:
import pandas as pd


# Generic functions to attain the data

In [12]:
def save_data():
    url = "https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz"
    filename = url.split("/")[-1]

    with open(filename, "wb") as f:
        r = requests.get(url)
        f.write(r.content)

    with gzip.open(filename, 'rb') as f_in:
        with open('movie_data.csv', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [13]:
def prepare_data_test(df):
    X_test = df.iloc[40000:42500]
    X_test['text'] = X_test['review']
    X_test['text'] = X_test['text']
    y_true = list(X_test['sentiment'])
    return X_test.drop(['sentiment','review'],axis=1), y_true

In [14]:
def prepare_data():
    df = pd.read_csv('movie_data.csv')
    return prepare_data_test(df)
save_data()
X_test, y_true = prepare_data()

/tmp/ipykernel_34/513715063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['text'] = X_test['review']
/tmp/ipykernel_34/513715063.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['text'] = X_test['text']


In [15]:
print(X_test)

                                                    text
40000  Such energy and vitality. You just can't go wr...
40001  but Thomas Ian Griffith just doesn't have the ...
40002  Why this film was only released in 4 states is...
40003  I'm not here to tell you "Armored" is Kubricki...
40004  Most college students find themselves lost in ...
...                                                  ...
42495  This movie was terrific and even with a less t...
42496  I've seen a lot of bad movies in my life. Date...
42497  Quite honestly, The Omega Code is the worst mo...
42498  "In 1927 a young man, alone in a single engine...
42499  For those expecting the cover art and story ou...

[2500 rows x 1 columns]


In [16]:
print(y_true)

[1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 

# Notebook authentication to use the model

In [17]:
from huggingface_hub import notebook_login

In [18]:
notebook_login()

# Zero Shot Text Classification

Pipeline preparation

In [19]:
from transformers import pipeline
from tqdm import tqdm

classifier = pipeline("zero-shot-classification", model="google/gemma-2b", device=0)

labels = ["positive", "negative"]

classifications = []

2024-05-08 10:53:07.712066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 10:53:07.712158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 10:53:07.822743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Classification of each review

In [20]:
for review in tqdm(X_test['text']):
    #print(review)
    
    classification = classifier(review, labels)
    classifications.append(classification)
    
    #print(classification)

# Now 'classifications' contains the predicted sentiment labels (0 or 1) for each element in x_test
# print(classifications)


  0%|          | 10/2500 [00:10<38:34,  1.08it/s]--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-pa

# Results analysis

In [22]:
results = []

for classification in classifications:
    if classification['labels'][0] == "positive":
        results.append(1)
    else:
        results.append(0)

print(results)


[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 

In [23]:
# Calculate accuracy
correct_predictions = sum(1 for pred, true in zip(results, y_true) if pred == true)
total_predictions = len(y_true)
accuracy = correct_predictions / total_predictions

print("Accuracy:", accuracy*100,"%")

Accuracy: 59.96 %


In [24]:
from sklearn.metrics import classification_report
report = classification_report(y_true, results)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.96      0.71      1268
           1       0.86      0.22      0.36      1232

    accuracy                           0.60      2500
   macro avg       0.71      0.59      0.53      2500
weighted avg       0.71      0.60      0.54      2500



These last two cells exist to clear resourses if the code was ran on Google Colab

In [ ]:
#!pip install numba

In [ ]:
#from numba import cuda
#device = cuda.get_current_device()
#device.reset()